In [0]:
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import cv2 as cv
import pandas as pd
import os
import scipy.io as sio

import tensorflow as tf
from keras import backend as K
from keras.models import Sequential
from keras.models import load_model, Model
from keras.layers import InputLayer, Lambda, Conv2D , Dense, Flatten, MaxPooling2D, Dropout, LeakyReLU, Concatenate, BatchNormalization

from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.utils import multi_gpu_model
from keras.utils import plot_model
from keras.layers.merge import concatenate

In [0]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [0]:
df_train = pd.read_csv("training.csv", index_col=0).sample(frac=1,random_state=1)
df_train.head()

In [0]:
df_train['x1'] = df_train['x1'].apply(lambda x: x/640)
df_train['y1'] = df_train['y1'].apply(lambda x: x/480)
df_train['x2'] = df_train['x2'].apply(lambda x: x/640)
df_train['y2'] = df_train['y2'].apply(lambda x: x/480)
df_train.head()

In [0]:
df_train['h'] = df_train['y2'] - df_train['y1']
df_train['w'] = df_train['x2'] - df_train['x1']
df_train['x1'] = (df_train['x1'] + df_train['x2']) / 2
df_train['y1'] = (df_train['y1'] + df_train['y2']) / 2
df_train = df_train.drop(['x2', 'y2'], axis = 1)
y_train = df_train.as_matrix()
df_train.head()

In [0]:
i = 0
num_img = 14000
data_train = np.zeros(shape=(num_img,480//4,640//4,3))

for img_name in df_train.index:
    if i >= num_img:
        break
    img  = cv.imread("train_img/"+img_name,1)
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    data_train[i] = cv.resize(img , (160,120))/255
    if i % 500 == 0:
        print("Progress : ", i/(num_img/100),"%")
    i +=1

In [0]:
plt.figure(figsize=(16,20))
for i in range(20):
    plt.subplot(5,4,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(data_train[i], cmap=plt.cm.binary)
    plt.grid('off')

In [0]:
with tf.device('/cpu:0'):
    visible = Input(shape=(120,160,3))


    #1
    conv1  = Conv2D(32, kernel_size=3,padding='same')(visible)
    batch1 = BatchNormalization()(conv1)
    leaky1 = LeakyReLU()(batch1)
    pool1  = MaxPooling2D(pool_size=(2, 2))(leaky1)

    #2
    conv2  = Conv2D(64, kernel_size=3,padding='same')(pool1)
    batch2 = BatchNormalization()(conv2)
    leaky2 = LeakyReLU()(batch2)
    pool2  = MaxPooling2D(pool_size=(2, 2))(leaky2)

    #3
    conv3  = Conv2D(128, kernel_size=3,padding='same')(pool2)
    batch3 = BatchNormalization()(conv3)
    leaky3 = LeakyReLU()(batch3)
    pool3  = MaxPooling2D(pool_size=(2, 2))(leaky3)

    #4
    conv4  = Conv2D(64, kernel_size=3,padding='same')(pool3)
    batch4 = BatchNormalization()(conv4)
    leaky4 = LeakyReLU()(batch4)

    #5
    conv5  = Conv2D(128, kernel_size=3,padding='same')(leaky4)
    batch5 = BatchNormalization()(conv5)
    leaky5 = LeakyReLU()(batch5)
    pool5 = MaxPooling2D(pool_size=(2, 2))(leaky5)
    
    #6
    conv6  = Conv2D(256, kernel_size=3,padding='same')(leaky5)
    batch6 = BatchNormalization()(conv6)
    leaky6 = LeakyReLU()(batch6)
    pool6 = MaxPooling2D(pool_size=(2, 2))(leaky6)

    #7
    conv7  = Conv2D(128, kernel_size=3,padding='same')(pool6)
    batch7 = BatchNormalization()(conv7)
    leaky7 = LeakyReLU()(batch7)

    #8
    conv8  = Conv2D(256, kernel_size=3,padding='same')(leaky7)
    batch8 = BatchNormalization()(conv8)
    leaky8 = LeakyReLU()(batch8)

    #9
    conv9  = Conv2D(512, kernel_size=3,padding='same')(leaky8)
    batch9 = BatchNormalization()(conv9)
    leaky9 = LeakyReLU()(batch9)

    #10
    conv10  = Conv2D(128, kernel_size=3,padding='same')(leaky9)
    batch10 = BatchNormalization()(conv10)
    leaky10 = LeakyReLU()(batch10)
    
    
    # branch 2
    #10
    conv11  = Conv2D(256, kernel_size=3,padding='same')(pool6)
    batch11 = BatchNormalization()(conv11)
    leaky11 = LeakyReLU()(batch11)
    
    #10
    conv12  = Conv2D(128, kernel_size=3,padding='same')(leaky11)
    batch12 = BatchNormalization()(conv12)
    leaky12 = LeakyReLU()(batch12)
    
    #merge
    concat1 = concatenate([leaky10, leaky12])
    
    conv13  = Conv2D(256, kernel_size=3,padding='same')(concat1)
    batch13 = BatchNormalization()(conv13)
    leaky13 = LeakyReLU()(batch13)
    
    #output2
    conv15  = Conv2D(20, kernel_size=3,padding='same')(leaky13)
    batch15 = BatchNormalization()(conv15)
    leaky16 = LeakyReLU()(batch15)
    
    flat2 = Flatten()(leaky16)
    hidden3 = Dense(128)(flat2)
    leaky17 = LeakyReLU()(hidden3)
    
    hidden4 = Dense(32)(leaky17)
    leaky18 = LeakyReLU()(hidden4)
    
    output2 = Dense(2)(leaky18)
    
    #output1
#     conv14  = Conv2D(128, kernel_size=3,padding='same')(leaky13)
#     batch14 = BatchNormalization()(conv14)
#     leaky14 = LeakyReLU()(batch14)
    
# #     concat2 = concatenate([leaky14, pool5])
    
#     conv25  = Conv2D(20, kernel_size=3,padding='same')(leaky14)
#     batch25 = BatchNormalization()(conv25)
#     leaky25 = LeakyReLU()(batch25)
    
#     flat1 = Flatten()(leaky25)
#     hidden1 = Dense(256)(flat1)
#     leaky15 = LeakyReLU()(hidden1)
    
#     hidden2 = Dense(64)(leaky15)
#     leaky20 = LeakyReLU()(hidden2)
    
#     concat2 = concatenate([leaky20, act1])
    
#     hidden5 = Dense(32)(concat2)
#     leaky21 = LeakyReLU()(hidden5)
    
#     output1 = Dense(2)(leaky21)
#     flat0 = Flatten()(leaky25)
#     dense0 = Dense(256)(flat0)
#     act0 = LeakyReLU()(dense0)
    
#     dense1 = Dense(64)(act0)
#     act1 = LeakyReLU()(dense0)
#     dense1 = Dense(32)(act1)
#     leaky21 = LeakyReLU()(dense1)
    
#     output1 = Dense(2)(leaky21)

    conv25  = Conv2D(20, kernel_size=3,padding='same')(leaky13)
    batch25 = BatchNormalization()(conv25)
    leaky26 = LeakyReLU()(batch25)
    
    flat22 = Flatten()(leaky26)
    hidden23 = Dense(128)(flat22)
    leaky27 = LeakyReLU()(hidden23)
    
    hidden24 = Dense(32)(leaky27)
    leaky28 = LeakyReLU()(hidden24)
    
    output1 = Dense(2)(leaky28)
    

    
#     #output3
#     conv16  = Conv2D(20, kernel_size=3,padding='same')(leaky5)
#     batch16 = BatchNormalization()(conv16)
#     leaky17 = LeakyReLU()(batch16)
    
#     flat3 = Flatten()(leaky17)
#     hidden5 = Dense(128)(fla3)
#     leaky19 = LeakyReLU()(hidden5)
    
#     hidden6 = Dense(32)(leaky19)
#     leaky20 = LeakyReLU()(hidden6)
    
#     output3 = Dense(2)(leaky20)
    

    


    model = Model(inputs=visible, outputs=[output1,output2])

In [0]:
import tensorflow.keras.backend as K1

In [0]:
def iou_coef(y_true, y_pred, smooth=1):
    xi1 = K1.maximum(y_true[0],y_pred[0])
    yi1 = K1.maximum(y_true[2],y_pred[2])
    xi2 = K1.minimum(y_true[1],y_pred[1])
    yi2 = K1.minimum(y_true[3],y_pred[3])
    inter_area = (xi2-xi1)*(yi2-yi1)
    
    
    box1_area = (y_true[1]-y_true[0])*(y_true[3]-y_true[2])
    box2_area = (y_pred[1]-y_pred[0])*(y_pred[3]-y_pred[2])
    union_area = box1_area + box2_area - inter_area
    
    iou = inter_area / union_area
    
    return iou

def iou_coef_loss(y_true, y_pred):
    return -iou_coef(y_true, y_pred)

In [0]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
model.fit(data_t, y, epochs=1,verbose =1, batch_size=128)

In [0]:
parallel_model = multi_gpu_model(model, gpus=4)

losses = {"output1": "mean_squared_error","output2": "mean_squared_error"}
parallel_model.compile(optimizer='rmsprop', loss="mean_squared_error",metrics=['accuracy'])

In [0]:
parallel_model.fit(data_train, [y_train[:,0:2],y_train[:,2:4]], epochs=20, batch_size=256,validation_split=0.05)

In [0]:
model.save('models/modelt1.h5')

In [0]:
p = parallel_model.predict(data_train[:500])

In [0]:
p , p2 = p

In [0]:
y_train

In [0]:
p

In [0]:
p2

In [0]:
i = 0
plt.figure(figsize=(20,80))
for i in range(500):
    img = np.copy(data_train[i])
    img = cv.rectangle(img, 
                       (int((y_train[i][0] - y_train[i][3]/2)*320*0.5) ,int((y_train[i][1] - y_train[i][2]/2)*240*0.5) ), 
                       (int((y_train[i][0] + y_train[i][3]/2)*320*0.5), int((y_train[i][1] + y_train[i][2]/2)*240*0.5)), 
                       (1,0,0), 1)
#     img = cv.rectangle(img, 
#                        (int((p[i][0] - p[i][3]/2)*320*0.5) ,int((p[i][1] - p[i][2]/2)*240*0.5) ), 
#                        (int((p[i][0] + p[i][3]/2)*320*0.5), int((p[i][1] + p[i][2]/2)*240*0.5)), 
#                        (0,1,0), 1)
#     img = cv.rectangle(img, 
#                        (int((p2[i][0] - p2[i][3]/2)*320) ,int((p2[i][1] - p2[i][2]/2)*240) ), 
#                        (int((p2[i][0] + p2[i][3]/2)*320), int((p2[i][1] + p2[i][2]/2)*240)), 
#                        (0,0,1), 2)
    img = cv.rectangle(img, 
                       (int((p[i][0] - p2[i][1]/2)*320*0.5) ,int((p[i][1] - p2[i][0]/2)*240*0.5) ), 
                       (int((p[i][0] + p2[i][1]/2)*320*0.5), int((p[i][1] + p2[i][0]/2)*240*0.5)), 
                       (0,0,1), 1)
    plt.subplot(10,2,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img)
    plt.grid('off')
    if i == 19:
        break
    i +=1
plt.show()   